In [ ]:
import pandas as pd

import mysql.connector

from sqlalchemy import create_engine

from skimpy import skim

# 0- importation des données en df

In [ ]:
username = 'postgres'
password = 'admin'
host = '127.0.0.1'  # ou l'adresse IP de votre serveur PostgreSQL
port = '5432'  # le port par défaut pour PostgreSQL est 5432
database = 'preventive_maintenance'

# Créer l'URL de connexion
db_url = f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}'

# Créer le moteur SQLAlchemy
engine = create_engine(db_url)

# Se connecter à la base de données
connexion = engine.connect()


In [ ]:
df_aero = pd.read_sql_query('''
                  SELECT * 
                  FROM aeronefs
                  ''', connexion)


In [ ]:
df_composants = pd.read_sql_query('''
                  SELECT * 
                  FROM composants
                  ''', connexion)

In [ ]:
df_degradations = pd.read_sql_query('''
                  SELECT * 
                  FROM degradations
                  ''', connexion)

In [ ]:
df_logs_vols = pd.read_sql_query('''
                  SELECT * 
                  FROM logs_vols
                  ''', connexion)

In [ ]:
connexion.close()

# 1- merge des dataframes

In [ ]:
# merge du degradations et composants en left join avec degradations en left

df_merge_aero_log = pd.merge(df_degradations, df_composants, how='inner', left_on='compo_concerned', right_on='ref_compo')

In [ ]:
# faire un group by par categoriee et ref_aero
df_merge_aero_log.groupby(['ref_aero', 'categorie' ])

In [ ]:
# merge du aeronefs et compoosants en left join avec aeronefs en left

df_merge_aero_log = pd.merge(df_aero, df_logs_vols, how='inner', left_on='ref_aero', right_on='aero_linked')

In [ ]:
# merge du df_merge_aero_compo et degradations en left join avec df_merge_aero_compo en left

df_merge_aero_log_degrad = pd.merge(df_merge_aero_log, df_degradations, how='inner', left_on='ref_aero', right_on='linked_aero')

In [ ]:
# merge du df_merge_aero_compo et degradations en left join avec df_merge_aero_compo en left

df_merge_total = pd.merge(df_merge_aero_log_degrad, df_composants, how='left', left_on='compo_concerned', right_on='ref_compo')

In [ ]:
df_merge_total.shape

In [ ]:
del df_aero, df_logs_vols, df_merge_aero_log, df_merge_aero_log_degrad, df_degradations, df_composants

# 2- Nettoayage du df

In [ ]:
df_merge_total.columns

In [ ]:
df_merge_total = df_merge_total.drop(columns=[
                            'aero_linked', 
                            'sensor_data',
                            'temp_unit',
                            'pressure_unit', 
                            'vibrations_unit',
                            'linked_aero', 
                            'compo_concerned',
                            'aero'])

In [ ]:
pd.set_option("display.max_columns", None)

In [ ]:
df_merge_total.head(3)

In [ ]:
# skim(df_merge_total)

In [ ]:
# suppression des colonne n'apportant pas d'informations
df_merge_total = df_merge_total.drop(columns=['need_replacement', 'end_maint'])

In [ ]:
# modification du type pour date 
df_merge_total['debut_service'] = pd.to_datetime(df_merge_total['debut_service'], format='%Y-%m-%d')
df_merge_total['last_maint'] = pd.to_datetime(df_merge_total['last_maint'], format='%Y-%m-%d')
df_merge_total['jour_vol'] = pd.to_datetime(df_merge_total['jour_vol'], format='%Y-%m-%d')
df_merge_total['measure_day'] = pd.to_datetime(df_merge_total['measure_day'], format='%Y-%m-%d')

In [ ]:
# modif type category
df_merge_total['etat_voyant'] = df_merge_total['etat_voyant'].astype('category')         
df_merge_total['categorie'] = df_merge_total['categorie'].astype('category') 

In [ ]:
# Ajout d'une colonne temps de vol cumulé depuis la derniere maintenance

# Assumons que df_merge_total est le DataFrame fourni et qu'il contient les colonnes 'temps_de_vol', 'last_maint', 'jour_vol', et 'ref_aero'
# Trier le DataFrame par avion et par date de vol
df_merge_total = df_merge_total.sort_values(by=['ref_aero', 'jour_vol']).reset_index(drop=True)

# Initialiser la colonne 'temps_de_vol_cumule' avec des zéros
df_merge_total['temps_de_vol_cumule'] = 0

# Parcourir chaque avion
for avion in df_merge_total['ref_aero'].unique():
    # Filtrer les données pour l'avion actuel
    df_avion = df_merge_total[df_merge_total['ref_aero'] == avion]
    
    # Variable pour stocker le temps de vol cumulé
    temps_de_vol_cumule = 0
    
    # Parcourir chaque ligne du DataFrame filtré pour l'avion actuel
    for i in df_avion.index:
        # Si la maintenance a eu lieu (en supposant que 'last_maint' est un booléen ou indicatif)
        if df_merge_total.loc[i, 'last_maint']:
            # Réinitialiser le temps de vol cumulé
            temps_de_vol_cumule = 0
        
        # Ajouter le temps de vol actuel au temps de vol cumulé
        temps_de_vol_cumule += df_merge_total.loc[i, 'time_en_air']
        
        # Mettre à jour la colonne 'temps_de_vol_cumule'
        df_merge_total.loc[i, 'temps_de_vol_cumule'] = temps_de_vol_cumule


        # affichage de la progression en %
        # print(f"Progression pour l'avion {avion}: {(i+1)/len(df_merge_total)*100:.2f}%")




In [ ]:
# skim(df_merge_total)

In [ ]:
df_merge_total.to_parquet('data_preclean_avion.parquet')